# Configuraciones iniciales

Desactivamos algunos warnings producto de librerías deprecadas

In [ ]:
import warnings

from typing import List

warnings.filterwarnings("ignore")

In [ ]:
# Dejar el path principal como el anterior.
import sys
sys.path.append('../')

Seteamos las configuraciones del Logging

In [ ]:
import logging

# Crear el logger
log = logging.getLogger(__name__)

# Setear el nivel del registro
log.setLevel(logging.WARNING)

# Formato de los mensajes
formatter = logging.Formatter("%(levelname)s: (%(asctime)s) [%(filename)s: %(lineno)s] %(message)s")

if not log.hasHandlers():
    # Handlers
    file_handler = logging.FileHandler("logging.log")
    file_handler.setFormatter(formatter)  # Setear el formato del handler
    # Agregar el handler al logger
    # log.addHandler(file_handler)

    stream_handler = logging.StreamHandler()
    stream_handler.setFormatter(formatter)
    # Agregar el handler al logger
    log.addHandler(stream_handler)

Instalar la librería plot-likert y otras librerías útiles

In [ ]:
# Librería para hacer gráficos Likert
# !pip install plot-likert

# Para obtener datos de excel
# !pip install openpyxl

# Para tener un transformador de data
# !pip install -U scikit-learn

# Para tener herramientas estadísticas
# !python -m pip install statsmodels

# Para tener Seaborn
# !pip install seaborn

Importamos algunas librerías útiles para el resto del notebook

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

## Creación de carpetas

Path para los datos

In [ ]:
DATA_PATH = Path(r"..\data")

if not DATA_PATH.exists():
    log.info(f"Creando carpeta {DATA_PATH = }")
    DATA_PATH.mkdir()

In [ ]:
# Path absoluto
QNA_PATH = DATA_PATH / "questions-and-answers.xlsx"

Path para las imágenes

In [ ]:
# Crear la carpeta de imágenes, si no se ha creado
from pathlib import Path


IMG_PATH = Path(r"..\img")

if not IMG_PATH.exists():
    log.info(f"Creando path {IMG_PATH = }")
    IMG_PATH.mkdir()

In [ ]:
QNA_COMPLETE_PATH = DATA_PATH / "questions-and-answers-complete.xlsx"
QNA_COMPLETE_PATH

In [ ]:
IMG_SUB_DIMS_COMPLETE_PATH = IMG_PATH / "sub-dims-complete"

if not IMG_SUB_DIMS_COMPLETE_PATH.exists():
    log.info(f"Creando path {IMG_SUB_DIMS_COMPLETE_PATH = }")
    IMG_SUB_DIMS_COMPLETE_PATH.mkdir()

In [ ]:
IMG_QUESTIONS_COMPLETE_PATH = IMG_PATH / "questions-complete"

if not IMG_QUESTIONS_COMPLETE_PATH.exists():
    log.info(f"Creando path {IMG_QUESTIONS_COMPLETE_PATH = }")
    IMG_QUESTIONS_COMPLETE_PATH.mkdir()

# Jugando con la librería `plot_likert`

Empezamos importando la librería para verificar que estuvo bien instalada.

In [ ]:
import plot_likert

Escogemos una paleta de colores apropiada para los gráficos Likert.

In [ ]:
sns.set_palette("coolwarm", n_colors=10)
pal = sns.color_palette()
pal

Hagamos un pequeño experimento con la librería Likert.

In [ ]:
rng = np.random.default_rng(seed=42)
data = pd.DataFrame(rng.choice(plot_likert.scales.agree, (10,2)), columns=['Q1','Q2'])

In [ ]:
plot_likert.plot_likert(
    data, plot_likert.scales.agree,
    colors=plot_likert.colors.default_with_darker_neutral
);

El gráfico se ve bastante bien, para datos pequeños.

# Importación de los datos de los alumnos y sus respuestas a las preguntas

Ahora que se sabe cómo graficar con Likert, empecemos importando el dataset y algunas variables que serán de utilidad para el resto del notebook.

In [ ]:
log.debug(f"Importando datos de {QNA_COMPLETE_PATH}")
df = pd.read_excel(QNA_COMPLETE_PATH)

df.head()

In [ ]:
questions_ = list(df.columns)[1:]
questions_

In [ ]:
import copy


log.debug("Renombrando columnas")
questions_copy = copy.copy(questions_)
for i in range(len(questions_)):
    questions_copy[i] = f"P{i+1}.: " + questions_[i]

df.columns = ["Prom Cs"] + questions_copy


df.head()

In [ ]:
questions_ = questions_copy

# Subdim 1.1
subdim_1_1 = [questions_.pop(0), questions_.pop(0), questions_.pop(0), questions_.pop(0), questions_.pop(0)]

# Subdim 1.2
subdim_1_2 = [questions_.pop(0), questions_.pop(0), questions_.pop(0), questions_.pop(0), questions_.pop(0)]

# Subdim 1.3
subdim_1_3 = [questions_.pop(0), questions_.pop(0), questions_.pop(0), questions_.pop(0), questions_.pop(0),
              questions_.pop(0)]

# Subdim 2.1
subdim_2_1 = [questions_.pop(0), questions_.pop(0)]

# Subdim 2.2
subdim_2_2 = [questions_.pop(0), questions_.pop(0)]

# Subdim 2.3
subdim_2_3 = [questions_.pop(0), questions_.pop(0), questions_.pop(0), questions_.pop(0)]

# Subdim 2.4
subdim_2_4 = [questions_.pop(0), questions_.pop(0), questions_.pop(0), questions_.pop(0), questions_.pop(0),
              questions_.pop(0), questions_.pop(0)]

# Concatenar todas las preguntas
questions = subdim_1_1 + subdim_1_2 + subdim_1_3 + subdim_2_1 + subdim_2_2 + subdim_2_3 + subdim_2_4
questions

# Función para generar gráficos Likert

In [ ]:
scales = [
    "Nunca",
    "Rara vez",
    "Ocasionalmente",
    "Casi siempre",
    "Siempre, en todas las clases"
]

Generamos una lista de colores específicos para este tipo de gráfico.

In [ ]:
colors = list(sns.color_palette().as_hex())
n_colors = len(colors)

In [ ]:
likert_colors: plot_likert.colors.Colors = [
    plot_likert.colors.TRANSPARENT,
    colors[-1],
    colors[-2],
    "silver",
    colors[1],
    colors[0],
]

In [ ]:
def personalised_plot_likert(
        df, questions: List[str],
        scales=tuple(scales),
        title=None, xlabel='Porcentaje de respuestas',
        save_path=None, save_config=None,
        **kwargs
):
    log.info(f"personalised_plot_likert: ({title=}, {xlabel=})")
    kwargs.setdefault("bar_labels", True)
    kwargs.setdefault("colors", likert_colors.copy())
    kwargs.setdefault("plot_percentage", True)
    log.debug(f"{kwargs=}")

    # Filtrando data frame
    new_df = df[questions]

    # Creando el gráfico
    log.debug("Creando plot con plot_likert")
    ax = plot_likert.plot_likert(new_df, scales, **kwargs)
    ax.xaxis.set_label_text(xlabel)
    ax.set_title(title)

    if save_path:
        log.info(f"Salvando figura en {save_path=}")
        save_config = save_config or dict()
        save_config.setdefault("dpi", 600)
        save_config.setdefault("bbox_inches", "tight")
        log.debug(f"{save_config=}")
        ax.get_figure().savefig(save_path, **save_config)
    return ax

personalised_plot_likert(df, questions=subdim_1_1)
plt.show()

# Gráficos pastel

Hacemos una función para los gráficos de pastel.

In [ ]:
def parse_question(question):
    """
    Dada una pregunta, le quita caracteres especiales, y los deja en formato con letras minusculas, y con un guion de separacion
    :param question:
    :return:
    """
    question = question.casefold()
    question = filter(lambda x: x.isalpha() or x.isnumeric() or x.isspace(), question)
    question = "".join(question).replace(" ", "-")
    return question

parse_question(subdim_1_1[0]), subdim_1_1[0]

In [ ]:
def pie_plot(
        df,
        question,
        scales=tuple(scales),
        kwargs_pie: dict = None,
        add_legend=False,
        legend_loc=(1.2, 0.7),
        width_wrap=70,
        save=False,
        save_path: str = None,
        save_config=None,
        threshold_percentage=0.05,
):
    log.info(f"pie_plot({question=}, {scales=}, {kwargs_pie=}, {legend_loc=}, {width_wrap=})")
    from collections import OrderedDict
    from textwrap import wrap

    # Respuestas de todos los alumnos dada una pregunta
    list_answers = df[question]

    # Empezaremos a contar el número de respuestas dada la escala de Likert
    log.debug("Contando el número de respuestas por escala")
    count_scales = OrderedDict((scale, 0) for scale in scales)
    for answer in list_answers:
        count_scales[answer] += 1
    count_scales = count_scales.values()
    log.debug(f"{count_scales = }")

    # Filtramos los valores que son iguales a cero
    x = np.array([count for count in count_scales if count > 0])
    x = x / sum(x)  # Sacar el porcentaje (entre 0 y 1)
    log.debug(f"{x = }")
    labels: List[str] = [scale for count, scale in zip(count_scales, scales) if count > 0]
    log.debug(f"{labels = }")
    colors: List[str] = [color for count, color in zip(count_scales, likert_colors[1:]) if count > 0]
    log.debug(f"{colors = }")

    # Seteamos un diccionario para los argumentos del gráfico de pastel, y seteamos valores por default
    kwargs_pie = kwargs_pie or dict()
    kwargs_pie.setdefault("x", x)
    kwargs_pie.setdefault("labels", labels)
    kwargs_pie.setdefault("colors", colors)
    kwargs_pie.setdefault("autopct", '%1.1f%%')
    kwargs_pie.setdefault("textprops", dict(size='large'))
    kwargs_pie.setdefault("startangle", 0)
    log.debug(f"{kwargs_pie = }")

    # Generamos el gráfico pastel
    log.debug("Generando el gráfico de pastel.")
    fig, ax = plt.subplots()
    patches, texts, autotexts = ax.pie(**kwargs_pie)

    # Cambiamos las letras de los porcentajes para que estén ennegrecidas y de color blanco
    log.debug("Cambiando las letras a color blanco.")
    for autotext, value in zip(autotexts, x):
        autotext.set(fontweight='bold', color='white')
        if value < threshold_percentage:
            autotext.set(text="")


    # Cambialos el lugar de la leyenda
    log.info("Agregando leyenda y título")
    if add_legend: plt.legend(loc=legend_loc)
    ax.set_title("\n".join(wrap(question, width_wrap)))

    if save:

        if not save_path:
            save_path = parse_question(question)
            save_path = IMG_PATH / (save_path + ".png")
            log.debug(f"{save_path = }")

        save_config = save_config or dict()
        save_config.setdefault("dpi", 600)
        save_config.setdefault("bbox_inches", "tight")
        log.debug(f"{save_config=}")

        log.info(f"Salvando figura en {save_path=}")
        ax.get_figure().savefig(save_path, **save_config)
        log.info(f"Figura salvada.")

    return patches, texts, autotexts

pie_plot(df, subdim_1_1[0], save=False, kwargs_pie=dict(startangle=0))

plt.show()

# Gráficos

Ahora que tenemos funciones para todos los tipos de gráficos, esta sección será exclusivamente para incluir los gráficos pedidos.

In [ ]:
personalised_plot_likert(
    df,
    questions=subdim_1_1,
    figsize=(6.4, 8),
    title="Sub-dimensión 1.1: Preparación de la clase",
    save_path=IMG_SUB_DIMS_COMPLETE_PATH / "sub-dim-1.1-preparacion-de-la-clase.png",
)
plt.show()

In [ ]:
personalised_plot_likert(
    df,
    questions=subdim_1_2,
    figsize=(6.4, 6),
    title="Sub-dimensión 1.2: Metodologías activas",
    save_path=IMG_SUB_DIMS_COMPLETE_PATH / "sub-dim-1.2-metodologias-activas.png",
)
plt.show()

In [ ]:
personalised_plot_likert(
    df,
    questions=subdim_1_3,
    figsize=(6.4, 9),
    title="Sub-dimensión 1.3: Evaluación de y para los aprendizajes",
    save_path=IMG_SUB_DIMS_COMPLETE_PATH / "sub-dim-1.3-evaluacion-de-y-para-los-aprendizajes.png",
)
plt.show()

In [ ]:
personalised_plot_likert(
    df,
    questions=subdim_2_1,
    title="Sub-dimensión 2.1: Aprendizaje mediado con tecnologías",
    save_path=IMG_SUB_DIMS_COMPLETE_PATH / "sub-dim-2.1-aprendizaje-mediado-con-tecnologías.png",
)
plt.show()

In [ ]:
personalised_plot_likert(
    df,
    questions=subdim_2_2,
    title="Sub-dimensión 2.2: Aprendizaje Directo",
    save_path=IMG_SUB_DIMS_COMPLETE_PATH / "sub-dim-2.2-aprendizaje-directo.png",
)
plt.show()

In [ ]:
personalised_plot_likert(
    df,
    questions=subdim_2_3,
    figsize=(6.4, 6),
    title="Sub-dimensión 2.3: Comunicación Pedagógica",
    save_path=IMG_SUB_DIMS_COMPLETE_PATH / "sub-dim-2.3-comunicacion-pedagogica.png",
)
plt.show()

In [ ]:
personalised_plot_likert(
    df,
    questions=subdim_2_4,
    figsize=(6.4, 11),
    title="Sub-dimensión 2.4: Interacciones en el Trabajo Colaborativo Inclusivo",
    save_path=IMG_SUB_DIMS_COMPLETE_PATH / "sub-dim-2.4-interacciones-en-el-trabajo-cColaborativo-inclusivo.png",
)
plt.show()

Hacemos lo mismo para los gráficos pastel

In [ ]:
for question in questions:
    pie_plot(df, question, save=True, save_path=IMG_QUESTIONS_COMPLETE_PATH / (parse_question(question) + ".png"), kwargs_pie=dict(startangle=0), threshold_percentage=0.05)